In [8]:
# %% step up
import os
import mne
import numpy as np


In [ ]:
subjs=['D0053','D0063','D0065','D0066','D0070','D0071','D0081','D0094']

In [ ]:
stat_type='pval'
con='Auditory'
contrast='ave'

match stat_type:
    case "zscore":
        fif_read = lambda f: mne.read_epochs(f, False, preload=True)
    case "power":
        fif_read = lambda f: mne.read_epochs(f, False, preload=True)
    case "significance":
        fif_read = mne.read_evokeds
    case "pval":
        fif_read = mne.read_evokeds

In [9]:
chs = []
data_lst = []
for i, subject in enumerate(subjs):
    
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
    bids_root = os.path.join(LAB_root,'BIDS-1.0_LexicalDecRepDelay','BIDS')
    subj_gamma_stats_dir = os.path.join(bids_root, "derivatives", "stats", subject)
    
    file_dir = os.path.join(subj_gamma_stats_dir,f'{con}_{stat_type}-{contrast}.fif')
    subj_dataset = fif_read(file_dir)
    
    subj_data = subj_dataset[0].data
    subj_chs = subj_dataset[0].ch_names
    labeled_chs = [f"{subject}_{ch}" for ch in subj_chs]
    
    data_lst.append(subj_data)
    chs.extend(labeled_chs)
    if i == 0:
        times = subj_dataset[0].times
    
data = np.concatenate(data_lst, axis=0)

Reading C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\stats\D0053\Auditory_pval-ave.fif ...
    Found the data of interest:
        t =    -500.00 ...    2990.00 ms (No comment)
        0 CTF compensation matrices available
        nave = 1 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
Reading C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\stats\D0063\Auditory_pval-ave.fif ...
    Found the data of interest:
        t =    -500.00 ...    2990.00 ms (No comment)
        0 CTF compensation matrices available
        nave = 1 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
Reading C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\stats\D0065\Auditory_pval-ave.fif ...
    Found the data of interest:
        t =    -500.00 ...    2990.00 ms (No 